In [12]:
#ファイルの読み込み
import pandas as pd
df1 = pd.read_csv('data_jun.csv')
df1.head()

,year,futoko_rate,stu_per_tch,classes,pass_rate,rate_OII,rate_Lve
0,1991,1.04,18.08,3150,4.2,0.004,1.33
1,1992,1.16,17.81,3150,5.0,0.004,1.39
2,1993,1.24,17.43,3150,6.0,0.006,1.40
3,1994,1.32,17.11,3150,8.0,0.004,1.37
4,1995,1.42,16.86,3150,8.8,0.004,1.37


In [16]:
#特徴量から'year', ’stu_per_tch’, 'rate_Lve'を削除
df2 = df1.drop(['year','stu_per_tch', 'rate_Lve'], axis = 1)

#標準化する
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(df2)
sc_df1 = sc.transform(df2)
sc_df1 = pd.DataFrame(sc_df1, columns = df2.columns)
sc_df1.head()

,futoko_rate,classes,pass_rate,rate_OII
0,-1.833918,1.176697,-1.459897,-0.962980
1,-1.695985,1.176697,-1.220285,-0.962980
2,-1.604029,1.176697,-0.920769,-0.696441
3,-1.512074,1.176697,-0.321738,-0.962980
4,-1.397130,1.176697,-0.082125,-0.962980


In [17]:
#多項式特徴量、交互作用特徴量を出してみる
from sklearn.preprocessing import PolynomialFeatures
sc_df2 = sc_df1.loc[:, 'classes':]
poly = PolynomialFeatures(degree = 2)
sc_df3 = poly.fit_transform(sc_df2)
poly.get_feature_names_out()

array(['1', 'classes', 'pass_rate', 'rate_OII', 'classes^2',
       'classes pass_rate', 'classes rate_OII', 'pass_rate^2',
       'pass_rate rate_OII', 'rate_OII^2'], dtype=object)

In [18]:
#先ほど出した多項式特徴量、交互作用特徴量をデータフレームにする
sc_df4 = pd.DataFrame(sc_df3, columns = ['1', 'classes', 'pass_rate', 'rate_OII', 'classes^2',
       'classes pass_rate', 'classes rate_OII', 'pass_rate^2',
       'pass_rate rate_OII', 'rate_OII^2'])
sc_df4.head()

,1,classes,pass_rate,rate_OII,classes^2,classes pass_rate,classes rate_OII,pass_rate^2,pass_rate rate_OII,rate_OII^2
0,1.0,1.176697,-1.459897,-0.962980,1.384615,-1.717856,-1.133136,2.131300,1.405852,0.927331
1,1.0,1.176697,-1.220285,-0.962980,1.384615,-1.435905,-1.133136,1.489095,1.175110,0.927331
2,1.0,1.176697,-0.920769,-0.696441,1.384615,-1.083466,-0.819500,0.847816,0.641261,0.485030
3,1.0,1.176697,-0.321738,-0.962980,1.384615,-0.378588,-1.133136,0.103515,0.309827,0.927331
4,1.0,1.176697,-0.082125,-0.962980,1.384615,-0.096637,-1.133136,0.006745,0.079085,0.927331


In [19]:
#'futoko_rate’を戻して'1'を削除
sc_df4['futoko_rate'] = sc_df1['futoko_rate']
sc_df4 = sc_df4.drop(['1'], axis = 1)
sc_df4.head()


,classes,pass_rate,rate_OII,classes^2,classes pass_rate,classes rate_OII,pass_rate^2,pass_rate rate_OII,rate_OII^2,futoko_rate
0,1.176697,-1.459897,-0.962980,1.384615,-1.717856,-1.133136,2.131300,1.405852,0.927331,-1.833918
1,1.176697,-1.220285,-0.962980,1.384615,-1.435905,-1.133136,1.489095,1.175110,0.927331,-1.695985
2,1.176697,-0.920769,-0.696441,1.384615,-1.083466,-0.819500,0.847816,0.641261,0.485030,-1.604029
3,1.176697,-0.321738,-0.962980,1.384615,-0.378588,-1.133136,0.103515,0.309827,0.927331,-1.512074
4,1.176697,-0.082125,-0.962980,1.384615,-0.096637,-1.133136,0.006745,0.079085,0.927331,-1.397130


In [20]:
#相関係数を出してみる
abs(sc_df4.corr()['futoko_rate']).sort_values(ascending = False)

futoko_rate           1.000000
rate_OII              0.817131
pass_rate rate_OII    0.750320
rate_OII^2            0.584084
classes^2             0.551789
classes               0.471835
classes rate_OII      0.470770
classes pass_rate     0.220245
pass_rate             0.067452
pass_rate^2           0.054991
Name: futoko_rate, dtype: float64

In [21]:
#特徴量と正解データに分割
col = ['rate_OII', 'classes^2']
x = sc_df4[col]
t = sc_df4['futoko_rate']

In [22]:
#訓練データとテストデータの分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,t, test_size = 0.2, random_state = 0)

#モデルを作成し学習させる
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train,y_train) 

LinearRegression()

In [23]:
#決定係数を出してみる
model.score(x_test, y_test)

0.6379012689116574